# General notebook preparation

In [84]:
# Standard library imports
import os
import sys
import yaml

In [85]:
# Set constants for environment
IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL_CONTAINER = 'HOMELAB' in os.environ
env = {
    'PROJECT_NAME': 'LVGL UI Detector',
    'PROJECT_TAG': 'lvgl-ui-detector',
    'IN_COLAB': IN_COLAB,
    'IN_LOCAL_CONTAINER': IN_LOCAL_CONTAINER,
    'DIRS': {},
    'FILES': {},
    'ENV': os.environ,
}
required_modules = ['ultralytics', 'clearml']

# Environment setups

In [86]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive') # Mount Google Drive
    env['DIRS']['root'] = os.path.join("/content/drive/MyDrive/project")
elif IN_LOCAL_CONTAINER:
    # Local container already comes with the required modules pre-installed
    env['DIRS']['root'] = "/usr/src"
else:
    # Local development environment needs to have the required modules installed
    env['DIRS']['root'] = os.path.curdir

In [87]:
# Import all dependencies
missing_deps = False
try:
    from ultralytics import YOLO
    from clearml import Task, TaskTypes, Dataset, StorageManager, Logger
    import yaml
except ImportError as e:
    print(f"Failed to import required dependencies: {e}")
    missing_deps = True
    print("Run the next cell to install the required dependencies and then restart the runtime.")

In [88]:
if missing_deps:
    %pip install -q {*required_modules}

In [89]:
# Show the used versions of the required modules
for module in required_modules:
    %pip show {module}

Name: ultralytics
Version: 8.1.47
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /home/rini-debian/git-stash/lvgl-ui-detector/.venv/lib/python3.11/site-packages
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Name: clearml
Version: 1.15.0
Summary: ClearML - Auto-Magical Experiment Manager, Version Control, and MLOps for AI
Home-page: https://github.com/allegroai/clearml
Author: ClearML
Author-email: support@clear.ml
License: Apache License 2.0
Location: /home/rini-debian/git-stash/lvgl-ui-detector/.venv/lib/python3.11/site-packages
Requires: attrs, furl, jsonschema, numpy, pathlib2, Pillow, psutil, pyjwt, pyparsing, python-

In [90]:
# Configure folders
env['DIRS']['data'] = os.path.join(env['DIRS']['root'], "datasets")

In [91]:
# Query ClearML for available datasets
datasets = Dataset.list_datasets(env['PROJECT_NAME'], partial_name="UI Randomizer", tags=[env['PROJECT_TAG']], only_completed=True)
# Store dataset filenames per dataset
env['DATASETS'] = {}
for dataset in datasets:
    env['FILES'][dataset['id']] = Dataset.get(dataset['id']).list_files("*.yaml")
    env['DATASETS'][dataset['id']] = dataset


In [92]:
# Debugging output

print(env['FILES'])
print(env['DATASETS'])

{'d1c4ec5c8f7e4492ab7b2a8d0b584c88': ['ui_randoms.yaml'], 'e7f05a856ca34cd88940c4b8774f4c45': ['ui_randoms.yaml'], 'e113a14fec574025a15a5c9868746c4b': ['dataset_upload_test.yaml'], 'c520737c392c47608ffd52f33fc593f6': ['dataset_upload_test.yaml'], '12f2a7ee7a7f45b8866d2c5f75e92413': ['dataset_upload_test.yaml'], '640fc2e67b3b4b498fe6d9b1da764911': ['dataset_upload_test.yaml'], 'cef7f5e021484b089a5eccf0c014d150': ['dataset_upload_test.yaml'], '28b1c17f1fef47a793f9ac086b2d6b10': ['dataset_upload_test.yaml'], 'abc76956f1de41b386fa805e8858c884': ['ui_widgets.yaml'], '326ea43fc9ff4e6db5209218b6020ca6': ['ui_widgets.yaml'], '172affc0c4bb4a369e75f685782b5ad8': ['ui_widgets.yaml']}
{'d1c4ec5c8f7e4492ab7b2a8d0b584c88': {'name': 'UI Randomizer - ui_randoms', 'created': datetime.datetime(2024, 4, 14, 11, 22, 26, 188000, tzinfo=tzutc()), 'project': 'LVGL UI Detector', 'id': 'd1c4ec5c8f7e4492ab7b2a8d0b584c88', 'tags': ['lvgl-ui-detector', 'random'], 'version': '1.0.0'}, 'e7f05a856ca34cd88940c4b8774f

# Helper functions

In [93]:
def download_dataset(id: str, overwrite: bool = True):
    dataset = Dataset.get(id)
    return dataset.get_mutable_local_copy(env['DIRS']['data'], overwrite=overwrite)

In [94]:
def fix_dataset_path(file: str, replacement_path: str):
    # Replace path in dataset file to match current environment
    with open(file, 'r+') as f:
        dataset_content = yaml.safe_load(f)
        dataset_content['path'] = replacement_path
        print(f"Original dataset:\n{dataset_content}")
        f.seek(0)
        yaml.dump(dataset_content, f)
        f.truncate()
        f.seek(0)
        print(f"Adjusted dataset:\n{f.read()}")
        return dataset_content

In [95]:
def training_task(model_variant: str, dataset_id: str, args: dict, project: str = "LVGL UI Detector"):
    # Download & modify dataset
    env['DIRS']['target'] = download_dataset(dataset_id)
    dataset_file = os.path.join(env['DIRS']['target'], env['FILES'][dataset_id][0])
    dataset_content = fix_dataset_path(dataset_file, env['DIRS']['target'])
    args['data'] = os.path.join(env['DIRS']['target'], env['FILES'][dataset_id][0])
    # Create a ClearML Task
    task = Task.init(
        project_name="LVGL UI Detector",
        task_name=f"Train {model_variant} ({env['DATASETS'][dataset_id]['name']})",
        task_type=TaskTypes.training,
    )
    # Log "model_variant" parameter to task
    task.set_parameter("model_variant", model_variant)
    task.connect_configuration(name="Dataset YAML", configuration=args['data'])
    task.connect_configuration(name="Dataset Content", configuration=dataset_content)

    # Load the YOLOv8 model
    model = YOLO(f'{model_variant}.pt')
    task.connect(args)

    # Train the model 
    # If running remotely, the arguments may be overridden by ClearML if they were changed in the UI
    try:
        results = model.train(**args)
    except Exception as e:
        raise e
    finally:
        task.close()
    return results

# Choose dataset

In [96]:
valid_choice = False
while not valid_choice:
    dataset_choice = input("Enter the dataset ID to download:")
    if dataset_choice in env['DATASETS'].keys():
        valid_choice = True
    else:
        print("Dataset ID not found, please try again.")

# Model training

In [97]:
# Training inputs
model_variant = "yolov8n"
args = dict(
    epochs=25, 
    imgsz=480
)
results = training_task(model_variant, dataset_choice, args)


Original dataset:
{'names': {0: 'arc', 1: 'bar', 2: 'button', 3: 'buttonmatrix', 4: 'calendar', 5: 'chart', 6: 'checkbox', 7: 'dropdown', 8: 'image', 9: 'imagebutton', 10: 'keyboard', 11: 'label', 12: 'led', 13: 'line', 14: 'list', 15: 'menu', 16: 'messagebox', 17: 'roller', 18: 'scale', 19: 'slider', 20: 'spangroup', 21: 'spinbox', 22: 'spinner', 23: 'switch', 24: 'table', 25: 'tabview', 26: 'textarea', 27: 'tileview', 28: 'window'}, 'path': '/home/rini-debian/git-stash/lvgl-ui-detector/datasets', 'test': 'images/test', 'train': 'images/train', 'val': 'images/val'}
Adjusted dataset:
names:
  0: arc
  1: bar
  2: button
  3: buttonmatrix
  4: calendar
  5: chart
  6: checkbox
  7: dropdown
  8: image
  9: imagebutton
  10: keyboard
  11: label
  12: led
  13: line
  14: list
  15: menu
  16: messagebox
  17: roller
  18: scale
  19: slider
  20: spangroup
  21: spinbox
  22: spinner
  23: switch
  24: table
  25: tabview
  26: textarea
  27: tileview
  28: window
path: /home/rini-debia

Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


Ultralytics YOLOv8.1.47 🚀 Python-3.11.2 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/rini-debian/git-stash/lvgl-ui-detector/datasets/ui_randoms.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

100%|██████████| 755k/755k [00:00<00:00, 2.28MB/s]

Overriding model.yaml nc=80 with nc=29

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               


...

      25/25      1.48G     0.8165      2.028      1.014         44        480: 100%|██████████| 5/5 [00:01<00:00,  4.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.72it/s]

                   all         16         48      0.573      0.598      0.619      0.514



25 epochs completed in 0.018 hours.
Optimizer stripped from /home/rini-debian/git-stash/lvgl-ui-detector/runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from /home/rini-debian/git-stash/lvgl-ui-detector/runs/detect/train2/weights/best.pt, 6.2MB

Validating /home/rini-debian/git-stash/lvgl-ui-detector/runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.11.2 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
Model summary (fused): 168 layers, 3011303 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.91it/s]


                   all         16         48      0.582      0.593      0.631      0.517
                   arc         16          3      0.652          1      0.995      0.913
                   bar         16          7       0.82      0.656       0.86      0.518
              checkbox         16          3      0.273      0.333      0.486       0.26
              dropdown         16          9      0.898      0.889      0.984      0.893
                 label         16          4          1          0          0          0
                   led         16          2          1      0.772      0.995      0.722
                roller         16          5      0.836        0.8      0.886      0.735
                 scale         16          3      0.458      0.667       0.71       0.71
                slider         16          1      0.198          1      0.199      0.139
               spinbox         16          1          0          0     0.0995     0.0995
                switc

In [98]:
results.box.map

0.5167399056845642

In [99]:
results.box.map50

0.6314966115185627

In [100]:
results.box.map75

0.5589333009373979

In [101]:
results.box.maps

array([     0.9132,     0.51756,     0.51674,     0.51674,     0.51674,     0.51674,     0.25993,     0.89302,     0.51674,     0.51674,     0.51674,           0,     0.72198,     0.51674,     0.51674,     0.51674,     0.51674,     0.73543,        0.71,      0.1393,     0.51674,      0.0995,     0.51674,     0.32558,
           0.88538,     0.51674,     0.51674,     0.51674,     0.51674])